In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2

import dlib
from PIL import Image

import face_alignment
from skimage import io
import copy
import json
import time
import os
import pickle
import ffmpeg

from natsort import natsorted, ns

from scipy.signal import medfilt
from scipy.ndimage.filters import uniform_filter1d

In [2]:
def rgb_load(img_dir, first_img=0, last_img=None, interval=1):
    # create list of jpg images in directory
    filenames = [f for f in os.listdir(img_dir) if f.split('.')[1] == 'jpg']
    filenames.sort()
    
    img_list = []
    
    # selecting images to analyse
    filenames=filenames[first_img:last_img:interval]
    
    for filename in filenames:
        img = dlib.load_rgb_image(img_dir + filename)
        img_list.append(img)
        
    return filenames, img_list

In [3]:
def thr_load(img_dir, first_img=0, last_img=None, interval=1):
    # create list of jpg images in directory
    filenames = [f for f in os.listdir(img_dir) if f.split('.')[1] == 'npy']
    
    filenames = natsorted(filenames, alg=ns.IGNORECASE)
#     filenames.sort()
        
    img_list = []
        
    # selecting images to analyse
    filenames=filenames[first_img:last_img:interval]
        
    for filename in filenames:
        img = np.load(img_dir + filename)
        img_list.append(img)
        
    return filenames, img_list

In [4]:
def resize_image(image, new_size=(128, 128)):
    img_resize = Image.fromarray(image)
    img_resize = img_resize.resize(new_size, Image.ANTIALIAS)
    img_resize_np = np.array(img_resize)
    return img_resize_np

In [5]:
def resize_image_list(image_list, new_size=(128, 128)):
    new_image_list = []
    for i, img in enumerate(image_list):
        new_image = resize_image(img, new_size=new_size)
        new_image_list.append(new_image)
    return new_image_list 

In [6]:
def json_load(json_dir):
    # create list of jpg images in directory
    filenames = [f for f in os.listdir(json_dir) if f.split('.')[1] == 'json']
    filenames.sort()
    
    json_list = []
    
    for filename in filenames:
        json_path = json_dir + filename
        
        with open(json_path) as json_file:
            json_data = json.load(json_file)
            json_list.append(json_data)
        
    return filenames, json_list

In [7]:
def json2tuples(json_lmarks):
    landmark_tuples_thr = []
    for c in json_lmarks['shapes']:
        x = int(c['points'][0][0])
        y = int(c['points'][0][1])
        landmark_tuples_thr.append((x,y))
    return landmark_tuples_thr

In [8]:
def json2tuples_list(json_list):
    lmark_list = []
    for json_lmarks in json_list:
        lmarks = json2tuples(json_lmarks)
        lmark_list.append(lmarks)
    return lmark_list

In [9]:
def compare_plot(rgb_img_list, thr_img_list, img_no):
    fig, ax = plt.subplots(ncols=2, figsize=(16, 12))
    ax[0].imshow(thr_img_list[img_no])
    ax[1].imshow(rgb_img_list[img_no])
    ax[0].grid()
    ax[1].grid()
    plt.show()

In [10]:
def sel_3d_lmarks(lmarks_list, sel_list):
    lmarks_sel_list = []
    for lmarks in lmarks_list:
        lmarks_sel = [lmarks[x] for x in sel_list] 
        lmarks_sel_list.append(lmarks_sel)
    return lmarks_sel_list

In [11]:
# output = optimal_quantization(mat, True)  
def optimal_quantization(t2d_data, print_mode = False):
    
    t2d_data_cp = copy.deepcopy(t2d_data)  
    
    vector_data= np.reshape(t2d_data_cp, t2d_data_cp.shape[0]*t2d_data_cp.shape[1])
    # select only values above zero when calculating min/max to avoid padding - sm
    vector_data = vector_data[vector_data > 0]
    
    min_T=np.percentile(vector_data, 2.5)  #2.5 percentile point
#     max_T=np.percentile(vector_data, 97.5) #97.5 percentile point
    
    # try a higher max_T - sm
    max_T=np.percentile(vector_data, 100) #99.5 percentile point
#     max_T = 36

    t2d_data_cp[np.where(t2d_data_cp > max_T)]=max_T
    t2d_data_cp[np.where(t2d_data_cp < min_T)]=min_T

    opt_T=min_T
    count=0
    while True:
        count+=1
        mean_back=np.mean(t2d_data_cp[np.where(t2d_data_cp <= opt_T)])
        mean_obj=np.mean(t2d_data_cp[np.where(t2d_data_cp > opt_T)])
        if np.abs(opt_T - (mean_back+mean_obj)/2 )<0.005:
            break;
        else:
            opt_T=(mean_back+mean_obj)/2

    min_T=opt_T

    if print_mode:
        print("optimal thermal range is [%f, %f]"%(min_T,max_T))

    t2d_data_cp[np.where(t2d_data_cp < min_T)]=min_T

    quantized_t_img=np.zeros((t2d_data_cp.shape[0],t2d_data_cp.shape[1]), np.uint8)

    for i in range(0, t2d_data_cp.shape[0]):
        for j in range(0, t2d_data_cp.shape[1]):
            quantized_t_img[i,j]=255*((t2d_data_cp[i,j]-min_T)/(max_T-min_T))

    return quantized_t_img 

In [12]:
def opt_quant_list(image_list, print_mode=False):
    new_image_list = []
    for i, img in enumerate(image_list):
        new_image = optimal_quantization(img, print_mode = print_mode)
        new_image_list.append(new_image)
    return new_image_list 

In [13]:
def output_thr_jpg_list(img_list, filename_list, save_dir):
    for i, img in enumerate(img_list):
        fname = filename_list[i].split('.')[0]
        save_path = save_dir + fname + '_proc.jpg'
        im = Image.fromarray(img)
        im = im.convert("L")
        im.save(save_path)

In [14]:
def landmark_error(gtruth_lms, pred_lms):
    K = len(gtruth_lms)
#     print('K:', K)
    leye_gt = gtruth_lms[0:2]
    reye_gt = gtruth_lms[2:4]
    leye_gt_centre = np.array(leye_gt).mean(axis=0)
    reye_gt_centre = np.array(reye_gt).mean(axis=0)
    Di = np.linalg.norm(leye_gt_centre-reye_gt_centre)
#     print('leye_gt_centre:', leye_gt_centre)
#     print('reye_gt_centre:', reye_gt_centre)
#     print('Di:', Di)
    dist_sum = 0
    for i, lms in enumerate(gtruth_lms):
        gt = np.array(gtruth_lms[i])
        pred = np.array(pred_lms[i])
        dist = np.linalg.norm(gt-pred)
        dist_sum += dist
#         print(i, dist)

    lm_err = dist_sum/(K*Di)
    # print('lm_err:', lm_err)
    return lm_err   

In [15]:
def landmark_error_list(gtruth_lms_list, pred_lms_list):
    lm_err_list = []
    for i in range(len(gtruth_lms_list)):
        lm_err = landmark_error(gtruth_lms_list[i], pred_lms_list[i])
        lm_err_list.append(lm_err)
    return lm_err_list

In [16]:
def trans_3Dto2D_calc(thr_2Dlmarks_list, 
                      rgb_3Dlmarks_list, 
                      rgb_3Dlmarks_list_sel,
                      camera_matrix,
                      dist_coeffs):
    
    # select every 30th of rgb sel landmarks to match thermal
    rgb_lmarks_list_sel_i30 = [rgb_3Dlmarks_list_sel[i] 
                                 for i in range(0,len(rgb_3Dlmarks_list_sel), 30)]
    
    rgb_sel_lmarks_2D_list = []
    
    success_list = []
    rotvec_list = []
    transvec_list = []
    
    rgb_68_lmarks_2D_med_list = []

    for i in range(len(thr_2Dlmarks_list)):
        # convert landmarks to numpy arrays
        thr_lmarks_np = np.array(thr_2Dlmarks_list[i], dtype="double")
        rgb_lmarks_sel_np = np.stack(rgb_lmarks_list_sel_i30[i])
                
        # solve projection
        (success, rot_vector, trans_vector) = cv2.solvePnP(rgb_lmarks_sel_np,
                                                           thr_lmarks_np, 
                                                           camera_matrix, 
                                                           dist_coeffs)
        
        # project selected 3D RGB points to 2D Thermal image
        rgb_lmarks_sel_2D_np, jacobian = cv2.projectPoints(rgb_lmarks_sel_np, 
                                                           rot_vector, 
                                                           trans_vector, 
                                                           camera_matrix, 
                                                           dist_coeffs)
        
        rgb_sel_lmarks_2D_list.append(rgb_lmarks_sel_2D_np)
        
        success_list.append(success)
        rotvec_list.append(rot_vector)
        transvec_list.append(trans_vector)

    return rgb_sel_lmarks_2D_list, success_list, rotvec_list, transvec_list        

In [17]:
def trans_median_calc(rotvec_list, transvec_list):
    
    rotx_med = np.median([rot[0][0] for rot in rotvec_list])
    roty_med = np.median([rot[1][0] for rot in rotvec_list])
    rotz_med = np.median([rot[2][0] for rot in rotvec_list]) 

    trnx_med = np.median([trn[0][0] for trn in transvec_list])
    trny_med = np.median([trn[1][0] for trn in transvec_list])
    trnz_med = np.median([trn[2][0] for trn in transvec_list])
    
    rot_vector_med = np.array([[rotx_med], [roty_med], [rotz_med]])
    trans_vector_med = np.array([[trnx_med], [trny_med], [trnz_med]])

    return rot_vector_med, trans_vector_med

In [18]:
def trans_3Dto2D_apply(rgb_lmarks_list, rot_vector, trans_vector, 
                       camera_matrix, dist_coeffs):
    
    rgb_lmarks_2D_list = []
    
    for i in range(len(rgb_lmarks_list)):
#         print(i)
        # convert landmarks to numpy arrays
        rgb_lmarks_np = np.stack(rgb_lmarks_list[i])
#         print(rgb_lmarks_np.shape)
        
        # project 68 3D RGB points to 68 2D Thermal image
        rgb_lmarks_2D_np, jacobian = cv2.projectPoints(rgb_lmarks_np, 
                                                       rot_vector, 
                                                       trans_vector, 
                                                       camera_matrix, 
                                                       dist_coeffs)
        
        rgb_lmarks_2D_list.append(rgb_lmarks_2D_np)
        
    return rgb_lmarks_2D_list    

In [19]:
def labelled_thr_jpg_output(thr_img_oq_list, rgb_68_lmarks_2D_list):
    labelled_img_list = []

    for i in range(len(thr_img_oq_list)):

        im_thr_cp = thr_img_oq_list[i].copy()

        for n in range(0,68):
            try:
                x=int(rgb_68_lmarks_2D_list[i][n][0][0])
                y=int(rgb_68_lmarks_2D_list[i][n][0][1])
                cv2.circle(im_thr_cp, (x, y), 1, (255, 255, 255), -1)
            except:
                pass
        
        labelled_img_list.append(im_thr_cp)  
    
    return labelled_img_list

In [26]:
def run_transform(p, rgb_first_img, rgb_last_img, rgb_int, thr_first_img, thr_last_img, thr_int, 
                  focal_length=5000):
    
    rgb_img_dir = '/Volumes/Ashur DT2/steven/P'+p+'/RGB/00000/'
    rgb_fname_list, rgb_img_list = rgb_load(img_dir=rgb_img_dir, 
                                            first_img=rgb_first_img, 
                                            last_img=rgb_last_img, 
                                            interval=rgb_int)

    thr_img_dir = '/Volumes/Ashur DT2/steven/P'+p+'/FLIR_HiRes/npy/'
    thr_fname_list, thr_img_list = thr_load(img_dir=thr_img_dir, 
                                            first_img=thr_first_img, 
                                            last_img=thr_last_img, 
                                            interval=thr_int)

    # resize rgb images
    rgb_img_list = resize_image_list(rgb_img_list, new_size=(455,256))

    # generate matching lists from rgb (25fps) to thermal (30fps)
    # get new indices to sample closest frame in rgb for each thermal frame
    new_interval = len(rgb_img_list)/len(thr_img_list)
    print('len(rgb_img_list)', len(rgb_img_list))
    print('len(thr_img_list)', len(thr_img_list))

    new_indices = [round(x*new_interval) for x in range(0,len(thr_img_list))]

    # create new rgb image list based on these indices
    rgb_img_list_30fps = [rgb_img_list[i] for i in new_indices]
    print('rgb img count:', len(rgb_img_list), 'to', len(rgb_img_list_30fps))

    # load thermal landmark picks
    json_dir = '/Volumes/Ashur DT2/steven/P'+p+'/FLIR_HiRes/picks_sel/'
    json_fnames, json_list = json_load(json_dir)
    # convert landmarks to list of tuples
    thr_lmark_list = json2tuples_list(json_list)

    # Load RGB 3D landmark picks
    with open('/Volumes/Ashur DT2/steven/pickles/p'+p+'_3Dlmarks_01_169.pickle', 'rb') as handle:
        rgb_3Dlmarks_list = pickle.load(handle)

    print('len(rgb_3Dlmarks_list)', len(rgb_3Dlmarks_list))

    # select first set of rgb landmarks for any frame  
    rgb_3Dlmarks_list = [x[0] for x in rgb_3Dlmarks_list]

    ### dirty fix for picked 3d landmarks less than 500. Just duplicate last landmark to end
    if len(rgb_3Dlmarks_list) < 500:
        for i in range(500 - len(rgb_3Dlmarks_list)):
            rgb_3Dlmarks_list.append(rgb_3Dlmarks_list[-1])

    print('len(rgb_3Dlmarks_list)', len(rgb_3Dlmarks_list))

    # resample RGB 3D landmarks to match thermal frames
    rgb_3Dlmarks_list_30fps = [rgb_3Dlmarks_list[i] for i in new_indices]
    print('rgb lmark count:', len(rgb_3Dlmarks_list), 'to', len(rgb_3Dlmarks_list_30fps))

#     if p in ['06', '10', '24']:
#         # select 9 key landmarks from 3d set
#         sel_list = [36,39,42,45,30,48,54,0,16]
#     else:
    # select 10 key landmarks from 3d set
    sel_list = [36,39,42,45,30,48,54,0,8,16]

    rgb_3Dlmarks_list_30fps_10 = sel_3d_lmarks(rgb_3Dlmarks_list_30fps, sel_list)        

    # set parameters for 3D to 2D transform
    size = thr_img_list[0].shape

    # Camera internals
#     focal_length = size[1]
#     focal_length = 5000

    center = (size[1]/2, size[0]/2)
    camera_matrix = np.array(
                             [[focal_length, 0, center[0]],
                             [0, focal_length, center[1]],
                             [0, 0, 1]], dtype = "double"
                             )

    # distortion coefficients - set to zero for now but 
    # would be useful to have values for this
    dist_coeffs = np.zeros((4,1))

    # Optimal quantisation
    thr_img_oq_list = opt_quant_list(thr_img_list)

    # calculate transform for the selected landmarks for the thermal labelled frames only
    (rgb_sel_lmarks_2D_list, 
     success_list, 
     rotvec_list, 
     transvec_list)  = trans_3Dto2D_calc(thr_lmark_list, 
                       rgb_3Dlmarks_list_30fps, 
                       rgb_3Dlmarks_list_30fps_10,
                       camera_matrix,
                       dist_coeffs)

#     # calculate 'median' rot and translation vectors from the list of transforms
#     rot_vector_med, trans_vector_med = trans_median_calc(rotvec_list, transvec_list)
    
    # more difficult to get an average of transformation than median of values
    # so just take first transform for now...
    
    # now choose the transform that makes the lowest error
    low_err_dict = {'03':15, '04':12, '05':17, '06':12, '07':8, '08':11, '09':6, '10':15, '13':7, 
                    '14':0, '15':18, '16':13, '17':7, '22':4, '24':11, '25':16, '26':10, '27':16}
    
    rot_vector_med = rotvec_list[low_err_dict[p]]
    trans_vector_med = transvec_list[low_err_dict[p]]

    rgb_lmarks_2D_list = trans_3Dto2D_apply(rgb_3Dlmarks_list_30fps, rot_vector_med, trans_vector_med, 
                                            camera_matrix, dist_coeffs)

    # output labelled thermal jpegs for QC
    labelled_img_list = labelled_thr_jpg_output(thr_img_oq_list, rgb_lmarks_2D_list)

    save_dir = '/Volumes/Ashur DT2/steven/P'+p+'/FLIR_HiRes/jpg_lmarks_169_low/'
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    output_thr_jpg_list(labelled_img_list, thr_fname_list, save_dir)

    # Save transformed landmarks
    with open('/Volumes/Ashur DT2/steven/pickles/p'+p+'_trans_lmarks_from10_low.pickle', 'wb') as handle:
        pickle.dump(rgb_lmarks_2D_list, handle)
        
    # output mp4 of thermal images with landmarks as QC
    jpg_path = '/Volumes/Ashur DT2/steven/P'+p+'/FLIR_HiRes/jpg_lmarks_169_low/'
    mp4_path = '/Volumes/Ashur DT2/steven/P'+p+'/FLIR_HiRes/mp4/'
    if not os.path.exists(mp4_path):
        os.makedirs(mp4_path)

    (
    ffmpeg.input(jpg_path + '*.jpg', pattern_type='glob', framerate=30)
          .output(mp4_path + 'p' + p + '_lmarks_169_from10_low.mp4').overwrite_output()
          .run(capture_stdout=True, capture_stderr=True)
    )

In [24]:
p = '03'

rgb_first_img = 175
rgb_last_img = 675
rgb_int = 1

thr_first_img = 1482
thr_last_img = 2082
thr_int = 1

start_time = time.time()
run_transform(p, rgb_first_img, rgb_last_img, rgb_int, 
              thr_first_img, thr_last_img, thr_int)
print("runtime:", time.time() - start_time)

len(rgb_img_list) 500
len(thr_img_list) 600
rgb img count: 500 to 600
len(rgb_3Dlmarks_list) 500
len(rgb_3Dlmarks_list) 500
rgb lmark count: 500 to 600
runtime: 137.86947202682495


In [27]:
p = '04'

focal_length = 5000

rgb_first_img = 11340
rgb_last_img = 11840
rgb_int = 1

thr_first_img = 12758
thr_last_img = 13358
thr_int = 1

start_time = time.time()
run_transform(p, rgb_first_img, rgb_last_img, rgb_int, 
              thr_first_img, thr_last_img, thr_int, focal_length)
print("runtime:", time.time() - start_time)

len(rgb_img_list) 500
len(thr_img_list) 600
rgb img count: 500 to 600
len(rgb_3Dlmarks_list) 500
len(rgb_3Dlmarks_list) 500
rgb lmark count: 500 to 600
runtime: 107.5644142627716


In [28]:
p = '05'

rgb_first_img = 14173
rgb_last_img = 14673
rgb_int = 1

thr_first_img = 16067
thr_last_img = 16667
thr_int = 1

focal_length = 5000

start_time = time.time()
run_transform(p, rgb_first_img, rgb_last_img, rgb_int, 
              thr_first_img, thr_last_img, thr_int, focal_length)
print("runtime:", time.time() - start_time)

len(rgb_img_list) 500
len(thr_img_list) 600
rgb img count: 500 to 600
len(rgb_3Dlmarks_list) 500
len(rgb_3Dlmarks_list) 500
rgb lmark count: 500 to 600
runtime: 95.61444306373596


In [29]:
p = '06'

rgb_first_img = 17500
rgb_last_img = 18000
rgb_int = 1

thr_first_img = 20278
thr_last_img = 20878
thr_int = 1

focal_length = 5000

start_time = time.time()
run_transform(p, rgb_first_img, rgb_last_img, rgb_int, 
              thr_first_img, thr_last_img, thr_int, focal_length)
print("runtime:", time.time() - start_time)

len(rgb_img_list) 500
len(thr_img_list) 600
rgb img count: 500 to 600
len(rgb_3Dlmarks_list) 500
len(rgb_3Dlmarks_list) 500
rgb lmark count: 500 to 600
runtime: 130.2781858444214


In [30]:
p = '07'

rgb_first_img = 10826
rgb_last_img = 11326
rgb_int = 1

thr_first_img = 12179
thr_last_img = 12779
thr_int = 1

focal_length = 5000

start_time = time.time()
run_transform(p, rgb_first_img, rgb_last_img, rgb_int, 
              thr_first_img, thr_last_img, thr_int, focal_length)
print("runtime:", time.time() - start_time)

len(rgb_img_list) 500
len(thr_img_list) 600
rgb img count: 500 to 600
len(rgb_3Dlmarks_list) 500
len(rgb_3Dlmarks_list) 500
rgb lmark count: 500 to 600
runtime: 135.69974899291992


In [31]:
p = '08'

rgb_first_img = 13008
rgb_last_img = 13508
rgb_int = 1

thr_first_img = 13098
thr_last_img = 13698
thr_int = 1

focal_length = 5000

start_time = time.time()
run_transform(p, rgb_first_img, rgb_last_img, rgb_int, 
              thr_first_img, thr_last_img, thr_int, focal_length)
print("runtime:", time.time() - start_time)

len(rgb_img_list) 500
len(thr_img_list) 600
rgb img count: 500 to 600
len(rgb_3Dlmarks_list) 500
len(rgb_3Dlmarks_list) 500
rgb lmark count: 500 to 600
runtime: 129.87158489227295


In [32]:
p = '09'

rgb_first_img = 13071
rgb_last_img = 13571
rgb_int = 1

thr_first_img = 14876
thr_last_img = 15476
thr_int = 1

focal_length = 5000

start_time = time.time()
run_transform(p, rgb_first_img, rgb_last_img, rgb_int, 
              thr_first_img, thr_last_img, thr_int, focal_length = 5000)
print("runtime:", time.time() - start_time)

len(rgb_img_list) 500
len(thr_img_list) 600
rgb img count: 500 to 600
len(rgb_3Dlmarks_list) 500
len(rgb_3Dlmarks_list) 500
rgb lmark count: 500 to 600
runtime: 138.511066198349


In [33]:
p = '10'

rgb_first_img = 15678
rgb_last_img = 16178
rgb_int = 1

thr_first_img = 16901
thr_last_img = 17501
thr_int = 1

focal_length = 5000

start_time = time.time()
run_transform(p, rgb_first_img, rgb_last_img, rgb_int, 
              thr_first_img, thr_last_img, thr_int, focal_length = 5000)
print("runtime:", time.time() - start_time)

len(rgb_img_list) 500
len(thr_img_list) 600
rgb img count: 500 to 600
len(rgb_3Dlmarks_list) 500
len(rgb_3Dlmarks_list) 500
rgb lmark count: 500 to 600
runtime: 132.5794939994812


In [34]:
p = '13'

rgb_first_img = 2743
rgb_last_img = 3243
rgb_int = 1

thr_first_img = 2063
thr_last_img = 2663
thr_int = 1

focal_length = 5000

start_time = time.time()
run_transform(p, rgb_first_img, rgb_last_img, rgb_int, 
              thr_first_img, thr_last_img, thr_int, focal_length = 5000)
print("runtime:", time.time() - start_time)

len(rgb_img_list) 500
len(thr_img_list) 600
rgb img count: 500 to 600
len(rgb_3Dlmarks_list) 484
len(rgb_3Dlmarks_list) 500
rgb lmark count: 500 to 600
runtime: 140.6541931629181


In [35]:
p = '14'

rgb_first_img = 18226
rgb_last_img = 18726
rgb_int = 1

thr_first_img = 20441
thr_last_img = 21041
thr_int = 1

focal_length = 5000

start_time = time.time()
run_transform(p, rgb_first_img, rgb_last_img, rgb_int, 
              thr_first_img, thr_last_img, thr_int, focal_length = 5000)
print("runtime:", time.time() - start_time)

len(rgb_img_list) 500
len(thr_img_list) 600
rgb img count: 500 to 600
len(rgb_3Dlmarks_list) 500
len(rgb_3Dlmarks_list) 500
rgb lmark count: 500 to 600
runtime: 145.24791073799133


In [36]:
p = '15'

rgb_first_img = 6950
rgb_last_img = 7450
rgb_int = 1

thr_first_img = 4051
thr_last_img = 4651
thr_int = 1

focal_length = 5000

start_time = time.time()
run_transform(p, rgb_first_img, rgb_last_img, rgb_int, 
              thr_first_img, thr_last_img, thr_int, focal_length = 5000)
print("runtime:", time.time() - start_time)

len(rgb_img_list) 500
len(thr_img_list) 600
rgb img count: 500 to 600
len(rgb_3Dlmarks_list) 500
len(rgb_3Dlmarks_list) 500
rgb lmark count: 500 to 600
runtime: 134.70751786231995


In [37]:
p = '16'

rgb_first_img = 3403
rgb_last_img = 3903
rgb_int = 1

thr_first_img = 3418
thr_last_img = 4018
thr_int = 1

focal_length = 5000

start_time = time.time()
run_transform(p, rgb_first_img, rgb_last_img, rgb_int, 
              thr_first_img, thr_last_img, thr_int, focal_length = 5000)
print("runtime:", time.time() - start_time)

len(rgb_img_list) 500
len(thr_img_list) 600
rgb img count: 500 to 600
len(rgb_3Dlmarks_list) 500
len(rgb_3Dlmarks_list) 500
rgb lmark count: 500 to 600
runtime: 137.35830307006836


In [38]:
p = '17'

rgb_first_img = 2661
rgb_last_img = 3161
rgb_int = 1

thr_first_img = 2355
thr_last_img = 2955
thr_int = 1

focal_length = 5000

start_time = time.time()
run_transform(p, rgb_first_img, rgb_last_img, rgb_int, 
              thr_first_img, thr_last_img, thr_int, focal_length = 5000)
print("runtime:", time.time() - start_time)

len(rgb_img_list) 500
len(thr_img_list) 600
rgb img count: 500 to 600
len(rgb_3Dlmarks_list) 500
len(rgb_3Dlmarks_list) 500
rgb lmark count: 500 to 600
runtime: 142.6355860233307


In [39]:
p = '22'

rgb_first_img = 7371
rgb_last_img = 7871
rgb_int = 1

thr_first_img = 7548
thr_last_img = 8148
thr_int = 1

focal_length = 5000

start_time = time.time()
run_transform(p, rgb_first_img, rgb_last_img, rgb_int, 
              thr_first_img, thr_last_img, thr_int, focal_length = 5000)
print("runtime:", time.time() - start_time)

len(rgb_img_list) 500
len(thr_img_list) 600
rgb img count: 500 to 600
len(rgb_3Dlmarks_list) 500
len(rgb_3Dlmarks_list) 500
rgb lmark count: 500 to 600
runtime: 148.0273141860962


In [40]:
p = '24'

rgb_first_img = 9442
rgb_last_img = 9942
rgb_int = 1

thr_first_img = 10086
thr_last_img = 10686
thr_int = 1

focal_length = 5000

start_time = time.time()
run_transform(p, rgb_first_img, rgb_last_img, rgb_int, 
              thr_first_img, thr_last_img, thr_int, focal_length = 5000)
print("runtime:", time.time() - start_time)

len(rgb_img_list) 500
len(thr_img_list) 600
rgb img count: 500 to 600
len(rgb_3Dlmarks_list) 500
len(rgb_3Dlmarks_list) 500
rgb lmark count: 500 to 600
runtime: 144.5041778087616


In [41]:
p = '25'

rgb_first_img = 754
rgb_last_img = 1254
rgb_int = 1

thr_first_img = 1819
thr_last_img = 2419
thr_int = 1

focal_length = 5000

start_time = time.time()
run_transform(p, rgb_first_img, rgb_last_img, rgb_int, 
              thr_first_img, thr_last_img, thr_int, focal_length = 5000)
print("runtime:", time.time() - start_time)

len(rgb_img_list) 500
len(thr_img_list) 600
rgb img count: 500 to 600
len(rgb_3Dlmarks_list) 500
len(rgb_3Dlmarks_list) 500
rgb lmark count: 500 to 600
runtime: 133.42524909973145


In [42]:
p = '26'

rgb_first_img = 1050
rgb_last_img = 1550
rgb_int = 1

thr_first_img = 352
thr_last_img = 952
thr_int = 1

focal_length = 5000

start_time = time.time()
run_transform(p, rgb_first_img, rgb_last_img, rgb_int, 
              thr_first_img, thr_last_img, thr_int, focal_length = 5000)
print("runtime:", time.time() - start_time)

len(rgb_img_list) 500
len(thr_img_list) 600
rgb img count: 500 to 600
len(rgb_3Dlmarks_list) 499
len(rgb_3Dlmarks_list) 500
rgb lmark count: 500 to 600
runtime: 163.84290289878845


In [43]:
p = '27'

rgb_first_img = 7996
rgb_last_img = 8496
rgb_int = 1

thr_first_img = 8767
thr_last_img = 9367
thr_int = 1

focal_length = 5000

start_time = time.time()
run_transform(p, rgb_first_img, rgb_last_img, rgb_int, 
              thr_first_img, thr_last_img, thr_int, focal_length = 5000)
print("runtime:", time.time() - start_time)

len(rgb_img_list) 500
len(thr_img_list) 600
rgb img count: 500 to 600
len(rgb_3Dlmarks_list) 500
len(rgb_3Dlmarks_list) 500
rgb lmark count: 500 to 600
runtime: 137.05243492126465
